<a href="https://colab.research.google.com/github/TechWithEva/Telecom-X-Challenge2-G9/blob/main/TELECOM_X_Challenge2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TELECOM X - Churn de Clientes
---
#### ***Problema:***  La empresa enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.
---



In [755]:
import pandas as pd
import requests
from numpy._core.defchararray import isnumeric
#

## Integración de Orígenes de Datos
< Extracción >

In [756]:
url = ('https://raw.githubusercontent.com/TechWithEva/Telecom-X-Challenge2-G9/refs/heads/main/rawdata/TelecomX_Data.json')
response=requests.get(url)
data=response.json()
base_ClientesTX = pd.DataFrame(data)
#base_ClientesTX.head()
#

## Tratamiento de Datos

#### Convertir a Archivo Plano y renombrar columnas

In [757]:
base_ClientesTX = pd.json_normalize(data)
columnasRenombre = {
    'customerID':'clienteID',
    'customer.gender':'genero',
    'customer.SeniorCitizen':'tercera_edad',
    'customer.Partner':'tiene_pareja',
    'customer.Dependents':'tiene_dependientes',
    'customer.tenure':'antiguedadContrato_Meses',
    'phone.PhoneService':'telefono_suscripcion',
    'phone.MultipleLines':'telefono_lineas_multiples',
    'internet.InternetService':'internet_suscripcion',
    'internet.OnlineSecurity':'seguridadOnline_suscripcion',
    'internet.OnlineBackup':'backupOnline_suscripcion',
    'internet.DeviceProtection':'proteccionDispositivos_suscripcion',
    'internet.TechSupport':'soporteTecnico_suscripcion',
    'internet.StreamingTV':'tvDigital_suscripcion',
    'internet.StreamingMovies':'peliculasDigitales_suscripcion',
    'account.Contract':'tipo_contrato',
    'account.PaperlessBilling':'factura_digital',
    'account.PaymentMethod':'metodo_pago',
    'account.Charges.Monthly':'cuota_mensual',
    'account.Charges.Total':'cargos_totales',
    'Churn':'churn'
    }
base_ClientesTX.rename(columns=columnasRenombre, inplace=True)

base_ClientesTX['pago_automatico']=base_ClientesTX['metodo_pago'].str.contains('automatic').astype(bool)
base_ClientesTX['tipo_servicio_internet']=base_ClientesTX['internet_suscripcion'].apply(lambda x: 'No Internet Service' if x=='No' else x)
base_ClientesTX['internet_suscripcion']=base_ClientesTX['internet_suscripcion'].apply(lambda x: False if x=='No' else True).astype(bool)
#

#### Tratamiento Datos No Esperados y Tipado

In [758]:
mapeoValoresStr = {
   'Female':'F',
   'Male':'M',
   'Mailed check':'Cheque Postal',
   'Electronic check':'Cheque Electronico',
   'Bank transfer (automatic)':'Transferencia Bancaria',
   'Credit card (automatic)':'Tarjeta Credito',
   'Month-to-month':'mensual',
   'One year':'anual',
   'Two year':'bienal',
   'Fiber optic':'Fibra Optica',
   'DSL':'DSL',
   'No Internet Service':'Sin suscripción a Internet',
   'Yes':True,
   'No':False,
   'No internet service':False,
   'No phone service':False
}
for col in base_ClientesTX.columns:
  if base_ClientesTX[col].dtype == 'object':
    if base_ClientesTX[col].unique()[0] in mapeoValoresStr:
      if base_ClientesTX[col].unique()[0] in ['Yes','No']:
        base_ClientesTX[col]= base_ClientesTX[col].map(mapeoValoresStr).astype(bool)
      else:
        base_ClientesTX[col] = base_ClientesTX[col].map(mapeoValoresStr).astype(str).str.strip().str.upper()

base_ClientesTX['tercera_edad']=base_ClientesTX['tercera_edad'].astype(bool)

base_ClientesTX['cargos_totales'] = pd.to_numeric(
    base_ClientesTX['cargos_totales'].str.strip().str.replace(',', '').replace(' ', ''),
    errors='coerce'
).fillna(0).astype(float)

base_ClientesTX['ambasSuscripciones'] = base_ClientesTX['telefono_suscripcion'] & base_ClientesTX['internet_suscripcion']
#

In [759]:
#Observar que hay en cada columna
for col in base_ClientesTX.columns:
  print('--------------------------------------------------')
  print(f'Valores únicos de {col.upper()} {base_ClientesTX[col].dtype} : {base_ClientesTX[col].nunique()}')
  print(f'{col}: {base_ClientesTX[col].unique()}')

print('--------------------------------------------------')
print(f'Longitud del código del cliente: {len(base_ClientesTX['clienteID'][0])} máscara "d4-c5"')
print('--------------------------------------------------')
#

--------------------------------------------------
Valores únicos de CLIENTEID object : 7267
clienteID: ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
--------------------------------------------------
Valores únicos de CHURN bool : 2
churn: [False  True]
--------------------------------------------------
Valores únicos de GENERO object : 2
genero: ['F' 'M']
--------------------------------------------------
Valores únicos de TERCERA_EDAD bool : 2
tercera_edad: [False  True]
--------------------------------------------------
Valores únicos de TIENE_PAREJA bool : 2
tiene_pareja: [ True False]
--------------------------------------------------
Valores únicos de TIENE_DEPENDIENTES bool : 2
tiene_dependientes: [ True False]
--------------------------------------------------
Valores únicos de ANTIGUEDADCONTRATO_MESES int64 : 73
antiguedadContrato_Meses: [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 

#### Tratamiento Duplicados - Nulos - Valores Faltantes - Vacíos

In [760]:
print(f'Cantidad de duplicados: {base_ClientesTX.duplicated().sum()}')
print(f'Cantidad de nulos: {base_ClientesTX.isnull().sum().sum()}')
print(f'Cantidad de vacíos: {base_ClientesTX.isna().sum().sum()}')
#print(f'Cantidad de Vacíos: {base_ClientesTX.apply(lambda x: x.astype(str).str.strip()=='').sum()}')

base_ClientesTX = base_ClientesTX[base_ClientesTX['churn']!='']
base_ClientesTX = base_ClientesTX[~base_ClientesTX['cargos_totales'].astype(str).str.isspace()]
print(f'Cantidad de Vacíos: {base_ClientesTX.apply(lambda x: x.astype(str).str.strip()=='').sum()}')
#base_ClientesTX.info()

Cantidad de duplicados: 0
Cantidad de nulos: 0
Cantidad de vacíos: 0
Cantidad de Vacíos: clienteID                             0
churn                                 0
genero                                0
tercera_edad                          0
tiene_pareja                          0
tiene_dependientes                    0
antiguedadContrato_Meses              0
telefono_suscripcion                  0
telefono_lineas_multiples             0
internet_suscripcion                  0
seguridadOnline_suscripcion           0
backupOnline_suscripcion              0
proteccionDispositivos_suscripcion    0
soporteTecnico_suscripcion            0
tvDigital_suscripcion                 0
peliculasDigitales_suscripcion        0
tipo_contrato                         0
factura_digital                       0
metodo_pago                           0
cuota_mensual                         0
cargos_totales                        0
pago_automatico                       0
tipo_servicio_internet         

In [761]:
base_ClientesTX.sample(50)

,clienteID,churn,genero,tercera_edad,tiene_pareja,tiene_dependientes,antiguedadContrato_Meses,telefono_suscripcion,telefono_lineas_multiples,internet_suscripcion,...,tvDigital_suscripcion,peliculasDigitales_suscripcion,tipo_contrato,factura_digital,metodo_pago,cuota_mensual,cargos_totales,pago_automatico,tipo_servicio_internet,ambasSuscripciones
935,1320-HTRDR,True,F,False,False,False,3,True,True,True,...,False,False,MENSUAL,True,CHEQUE ELECTRONICO,75.50,220.60,False,FIBRA OPTICA,True
2679,3721-CNZHX,False,M,False,False,False,15,True,False,False,...,False,False,ANUAL,False,CHEQUE POSTAL,19.80,304.60,False,SIN SUSCRIPCIÓN A INTERNET,False
1146,1625-JAIIY,True,F,False,True,True,68,True,True,True,...,True,False,ANUAL,True,CHEQUE ELECTRONICO,83.00,5685.80,False,FIBRA OPTICA,True
6333,8714-CTZJW,False,F,False,False,False,4,True,False,True,...,False,True,MENSUAL,True,TARJETA CREDITO,82.85,319.60,True,FIBRA OPTICA,True
7126,9804-ICWBG,True,M,False,False,False,1,True,False,True,...,False,False,MENSUAL,True,CHEQUE ELECTRONICO,69.90,69.90,False,FIBRA OPTICA,True
4182,5734-EJKXG,False,F,False,False,False,61,True,True,True,...,True,True,ANUAL,True,CHEQUE ELECTRONICO,118.60,7365.70,False,FIBRA OPTICA,True
60,0103-CSITQ,False,F,False,True,False,57,True,True,True,...,True,True,ANUAL,True,TRANSFERENCIA BANCARIA,109.40,6252.70,True,FIBRA OPTICA,True
6200,8510-TMWYB,False,F,False,True,True,54,True,True,True,...,False,False,BIENAL,True,TRANSFERENCIA BANCARIA,59.80,3246.45,True,DSL,True
4527,6203-HBZPA,False,M,False,False,False,52,False,False,True,...,True,True,BIENAL,False,TARJETA CREDITO,63.90,3334.95,True,DSL,False
6926,9541-ZPSEA,False,M,False,True,True,68,True,False,True,...,False,False,BIENAL,False,TARJETA CREDITO,80.35,5375.15,True,FIBRA OPTICA,True


## Análisis de Datos

## Viz

## Conclusiones y Recomendaciones